In [1]:
import os
import pandas as pd
import numpy as np
import random
import re

In [ ]:
BASE_DIR = os.getcwd()
DATA_DIR = '/opt/ml/data/'
random.seed(42)
data = pd.read_csv(DATA_DIR+'/train.csv')

## Preprocessing

#### Remove any character that is not an uppercase letter, lowercase letter, digit, or Korean, or Chinese characters

In [86]:
noise = set()
special = re.compile(r'[^ A-Za-z0-9가-힣一-龥]')

In [87]:
noise_data = pd.read_csv('/opt/ml/data/noise_data.csv')
denoised_data = pd.read_csv('/opt/ml/data/denoised_text.csv')
original_text = pd.read_csv('/opt/ml/data/original_text.csv')

In [88]:
denoise_and_noise=denoised_data.copy()
for data in noise_data.itertuples():
    prob = random.uniform(0, 1)
    if prob >= 0.6:
        denoise_and_noise=pd.concat([denoise_and_noise, pd.DataFrame({'ID':[data.ID],'text':[data.text],'target':[data.target], 'url':[data.url],'date':[data.date]})], ignore_index=True)
denoise_and_noise = denoise_and_noise.drop_duplicates(['ID'], keep='last')
denoise_and_noise = denoise_and_noise.sample(frac=1).reset_index(drop=True)
denoise_and_noise.to_csv('/opt/ml/data/denoised_text+noise.csv', index=False)

In [ ]:
texts = []
for text in original_text.text:
    texts.append(special.sub('',text))

original_text_no_special = pd.DataFrame({'ID':original_text.ID,'text':texts,'target':original_text.target, 'url':original_text.url,'date':original_text.date})
original_text_no_special.to_csv('/opt/ml/data/original_text_no_special.csv', index=False)

#### Remove stopwords and punctuation from the chatGPT augmented data

In [3]:
augData=pd.read_csv('../data/aug_data.csv')
for i in range(len(augData)):
    line=augData.loc[i,'text'].replace('"','').replace('...',',').replace('..',',').replace('..',',').replace('·',',')
    line=re.sub("(ㆍ|…)",',',line)
    line=re.sub("(‘|’|')","",line)
    line=re.sub(" 등 "," ",line)
    augData.loc[i,'text']=line

augData.to_csv("augData_news.csv",index=False)   
augData.tail()

#### Remove stopwords and punctuation from the original data

In [ ]:
noNoise=pd.read_csv('./no_noise_data.csv')

#drop columns except text and target 
noNoise=noNoise.drop(columns=['url','date','ID','Unnamed: 0'])
for i in range(len(noNoise)):
    line=noNoise.loc[i,'text'].replace('...','').replace('·',',')
    line=re.sub("(종합| 등 |게시판 |종합2보|…)"," ",line)
    noNoise.loc[i,'text']=line
    
noNoise.to_csv("news_pre.csv",index=False)
noNoise.tail()

##### Other replaceable words and stopwords

In [ ]:
#line=re.sub("↑"," 증가",line)
#line=line.replace('中','중국 ')
#line=line.replace('美','미국 ')
#line=line.replace('韓','한국 ')
#line=line.replace('北','북한 ')
#line=line.replace('日','일본 ')
#line=line.replace('英','영국 ')
#line=re.sub("(종합| 등 |게시판 |종합|종합2보|…| 첫 |역대 | 새 | 연속 |작년 |올해 |최대 )"," ",line)